In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import nltk
from gensim.models import Word2Vec
import numpy as np

In [3]:

train_df = pd.read_csv('../res/data_all_train.csv', header=0)
test_df = pd.read_csv('../res/data_all_test.csv', header=0)
validation_df = pd.read_csv('../res/data_all_validate.csv', header=0)

In [4]:
documents = []
labels=[]
for label in train_df.subreddit:
    labels.append(label)
for doc in train_df.text:
    unicode_string = nltk.word_tokenize(doc.decode('utf8'))
    documents.append(unicode_string)
    
print(len(documents))

model = Word2Vec(documents, size=80)

model.train(documents, total_examples=len(documents), epochs=20)


8189


4468460

In [5]:
twod_doc_array=[]
for doc in documents:
    word_array=[]
    i=0
    while(i<len(doc) ):
        if doc[i] in model.wv.vocab:
            word_array.append(list(model[doc[i]]))
        i+=1
    i=0
    while (i<40):
        word_array.append(np.zeros(80))
        i+=1
    
    twod_doc_array.append(word_array[:40])

In [6]:
twod_doc_array=np.array( twod_doc_array)

print twod_doc_array.shape
x=np.reshape(twod_doc_array,(twod_doc_array.shape[0],twod_doc_array.shape[1],twod_doc_array.shape[2],1))
print x.shape

(8189, 40, 80)
(8189, 40, 80, 1)


In [7]:
def cnn_model_fn(features, labels, mode):
    
    # Input Layer

    input_layer = tf.reshape(features['x'], [-1, 40, 80, 1])
    input_layer =tf.cast(input_layer,dtype=tf.float32)

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 10 * 20 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(1e-3)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
labellist=list(set(labels))

In [9]:
print labellist

['gameofthrones', 'dota2', 'conservative', 'leagueoflegends', 'askscience', 'libertarian', 'comicbooks', 'explainlikeimfive', 'thewalkingdead', 'anime']


In [10]:
y=[]
for i in labels:
    y.append(labellist.index(i))
print len(y)
y=np.array(y)
x=np.array(x)

8189


In [11]:
import time

In [12]:
start = time.time()

In [13]:
classifier = tf.estimator.Estimator(
model_fn=cnn_model_fn, model_dir="/tmp/dummy_model2")


# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={'x':x},
  y=y,
  batch_size=100,num_epochs=None,
  shuffle=True)

classifier.train(input_fn=train_input_fn,steps=10000)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/dummy_model2/model.ckpt.
INFO:tensorflow:loss = 2.23601, step = 1
INFO:tensorflow:global_step/sec: 0.68197
INFO:tensorflow:loss = 2.34049, step = 101 (146.636 sec)
INFO:tensorflow:global_step/sec: 0.589587
INFO:tensorflow:loss = 2.20898, step = 201 (169.611 sec)
INFO:tensorflow:global_step/sec: 0.720077
INFO:tensorflow:loss = 2.15921, step = 301 (138.874 sec)
INFO:tensorflow:Saving checkpoints for 375 into /tmp/dummy_model2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.454332
INFO:tensorflow:loss = 1.74466, step = 401 (220.105 sec)
INFO:tensorflow:global_step/sec: 0.322223
INFO:tensorflow:loss = 2.22397, step = 501 (310.344 sec)
INFO:tensorflow:Saving checkpoints for 601 into /tmp/dummy_model2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.46035
INFO:tensorflow:loss = 2.45159, step = 601 (217.224 sec)
INFO:tensorflow:glob

In [14]:
end=time.time()
print end-start
start=time.time()

17368.049932


In [15]:

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": x},
  y=y,
  num_epochs=1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-12-07-06:51:24
INFO:tensorflow:Restoring parameters from /tmp/dummy_model2/model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2017-12-07-06:52:02
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.253389, global_step = 10000, loss = 3.5207
{'loss': 3.5206952, 'global_step': 10000, 'accuracy': 0.2533887}


In [16]:
end=time.time()
print end-start

39.0795850754


In [17]:
testdocuments = []
testlabels=[]
for label in test_df.subreddit:
    testlabels.append(label)
for doc in test_df.text:
    unicode_string = nltk.word_tokenize(doc.decode('utf8'))
    testdocuments.append(unicode_string)

In [18]:
test_twod_doc_array=[]
for doc in testdocuments:
    word_array=[]
    i=0
    while(i<len(doc) ):
        if doc[i] in model.wv.vocab:
            word_array.append(list(model[doc[i]]))
        i+=1
    i=0
    while (i<40):
        word_array.append(np.zeros(80))
        i+=1
    
    test_twod_doc_array.append(word_array[:40])

test_twod_doc_array=np.array( test_twod_doc_array)
print len(testdocuments)
print test_twod_doc_array.shape
testx=np.reshape(test_twod_doc_array,(test_twod_doc_array.shape[0],test_twod_doc_array.shape[1],test_twod_doc_array.shape[2],1))
print testx.shape

2345
(2345, 40, 80)
(2345, 40, 80, 1)


In [19]:
test_y=[]
for i in testlabels:
    test_y.append(labellist.index(i))
print len(test_y)
test_y=np.array(test_y)
testx=np.array(testx)

2345


In [20]:
print test_y.shape

(2345,)


In [21]:
print testx.shape

(2345, 40, 80, 1)


In [22]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": testx},
  y=test_y,
  num_epochs=1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-12-07-06:55:24
INFO:tensorflow:Restoring parameters from /tmp/dummy_model2/model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2017-12-07-06:55:36
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.252878, global_step = 10000, loss = 3.59217
{'loss': 3.5921738, 'global_step': 10000, 'accuracy': 0.25287846}
